# 4. Shortest paths

**This file is used to prepare the final table from "table_graph" and then to build the algorithm to find the shortest path.**

In [ ]:
# imports
import pandas as pd
import datetime as dt
import numpy as np
from math import cos, sin, pi, sqrt, atan2

In [ ]:
!git lfs pull

## 4.1. Build df_public_transp from table_graph

In [ ]:
# Get the table for the graph 
table_graph = pd.read_csv('../data/table_graph.csv')
table_graph.columns = table_graph.columns.str[12:]
table_graph.head()

In [ ]:
# Convert column types
table_graph = table_graph.astype({'stop_sequence':int})
table_graph['arrival_time'] = table_graph['arrival_time'].apply(lambda x: dt.datetime.strptime(x, '%H:%M:%S'))
table_graph['departure_time'] = table_graph['departure_time'].apply(lambda x: dt.datetime.strptime(x, '%H:%M:%S'))
table_graph.dtypes

In [ ]:
table_graph

In [ ]:
# Filter and adjust table_graph

table_graph = table_graph.dropna(subset = ['trip_id'])
table_graph = table_graph[table_graph["stop_id"].str.contains("Parent")==False]
table_graph['stop_id'] = table_graph['stop_id'].str[:7]
table_graph = table_graph.drop_duplicates().reset_index(drop=True)

table_graph.shape

In [ ]:
# Construct dataframe with public transport to do the graph after

# empty table to fill with the edges
name_src = list(table_graph.add_prefix('src_').columns)
name_dst = list(table_graph.add_prefix('dst_').columns)
df_public_transp = pd.DataFrame(columns = name_src+name_dst+['duration'])

# find edges
for key, df_trip in table_graph.groupby(['trip_id']):
    
    df_trip = df_trip.drop_duplicates()
    
    if df_trip.empty or len(df_trip) == 1:
        continue

    df_trip = df_trip.sort_values(by="stop_sequence")
    sequence_pairs = zip(df_trip.stop_sequence.values[:-1], df_trip.stop_sequence.values[1:])
    
    for seq1,seq2 in sequence_pairs:
        df_src = df_trip[df_trip.stop_sequence==seq1].reset_index(drop=True)
        df_dst = df_trip[df_trip.stop_sequence==seq2].reset_index(drop=True)
        
        df_src_dst = pd.concat([df_src.add_prefix('src_'), df_dst.add_prefix('dst_')], axis=1)
        df_src_dst['duration'] = df_src_dst.dst_arrival_time - df_src_dst.src_departure_time
        df_src_dst['duration'] = df_src_dst['duration'].apply(lambda x: int(x.seconds/60))

        df_public_transp = pd.concat([df_public_transp, df_src_dst], ignore_index=True)

In [ ]:
# Filter non useful columns
df_public_transp.drop(columns=['dst_monday', 'dst_tuesday', 'dst_wednesday', 'dst_thursday', 'dst_friday', 'dst_vehicule'], inplace=True)
df_public_transp.rename(columns={"src_monday": "monday", "src_tuesday": "tuesday", "src_wednesday": "wednesday", "src_thursday": "thursday", "src_friday": "friday", "src_vehicule": "vehicule"}, inplace=True)

In [ ]:
df_public_transp

In [ ]:
# Save
df_public_transp.to_csv('../data/public_transp.csv', index=False)

## 4.2. Add walking edges

In [ ]:
# Get the table of all filtered nodes
table_stops = pd.read_csv('../data/stops.csv')
table_stops.columns = table_stops.columns.str[15:]
table_stops.head()

In [ ]:
# Find tuples of node for walking edges

p = pi/180 #F.radians
v = 50e-3

R = 6371 # Earth's radius in km

walking_nodes = []

for i, row_stop1 in table_stops.iterrows():
    for j, row_stop2 in table_stops.iterrows():
        
        stop_id_1 = row_stop1.stop_id
        stop_id_2 = row_stop2.stop_id

        lat1, lon1 = p*row_stop1.stop_lat, p*row_stop1.stop_lon
        lat2, lon2 = p*row_stop2.stop_lat, p*row_stop2.stop_lon

        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        distance = R * c  

        if (distance<0.5) & (stop_id_1!=stop_id_2):
            #for departure_time in range(60*6,60*18):
            walking_nodes.append((stop_id_1, stop_id_2, 0, distance/v))

In [ ]:
# Add walking edges to a dataframe
df_walking = pd.DataFrame(walking_nodes, columns=['src_stop_id','dst_stop_id','src_departure_time','duration'])
df_walking['src_trip_id'] = 'foot'
df_walking['vehicule'] = 'foot'
df_walking["duration"] = df_walking["duration"].apply(lambda x: int(x))

df_walking = df_walking.drop_duplicates()

In [ ]:
df_walking

In [ ]:
# Keep only stops that do not have parents i.e. is a Parent
df_walking_small = df_walking[df_walking["src_stop_id"].str.contains("Parent")==False]
df_walking_small = df_walking_small[df_walking_small["dst_stop_id"].str.contains("Parent")==False]

# Leave out the platform information: keep only the stop_id for each stop
df_walking_small['dst_stop_id'] = df_walking_small['dst_stop_id'].str[:7]
df_walking_small['src_stop_id'] = df_walking_small['src_stop_id'].str[:7]

# Drop duplicate edges 
df_walking_small = df_walking_small.drop_duplicates().reset_index(drop=True)

In [ ]:
# Save
df_walking_small.to_csv('../data/walk_small.csv', index=False)

In [ ]:
# Add time (intervall of 1 min)
timestamps = pd.date_range(start=dt.datetime.strptime('05:00:00', '%H:%M:%S'), end=dt.datetime.strptime('23:00:00', '%H:%M:%S'), freq='min')
df_walking_small['src_departure_time'] = [timestamps.tolist() for _ in range(len(df_walking_small))]
df_walking_edges = df_walking_small.explode('src_departure_time')
df_walking_edges['dst_arrival_time'] = df_walking_edges.apply(lambda row: row['src_departure_time'] + pd.to_timedelta(row['duration'], unit='minutes'), axis=1)

In [ ]:
# Add days
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday']
for col in days:
    df_walking_edges[col] = 1

In [ ]:
df_walking_edges

In [ ]:
# Save
df_walking_edges.to_csv('../data/walk_edges.csv', index=False)

## 4.3. Merge to get df_graph

In [ ]:
# Merge df_public_transp and df_walking_edges in a final graph
df_graph = pd.concat([df_public_transp, df_walking_edges], ignore_index=True)
df_graph.drop(columns=['src_arrival_time', 'src_stop_sequence', 'src_direction_id', 'dst_trip_id', 'dst_departure_time', 'dst_stop_sequence', 'dst_direction_id'], inplace=True)
df_graph.rename(columns={"src_trip_id": "trip_id"}, inplace=True)
df_graph

In [ ]:
# Save
df_graph.to_csv('../data/all_edges.csv', index=False)

## 4.4. Testing routing algorithm and probabilities

In [ ]:
# CHOOSE INPUTS
TRAVEL_DAY = 'monday'
ARRIVAL_TIME = '12:43:00'
PROBA_THRESHOLD = 0.95
DEPARTURE = '8503104'
DESTINATION = '8503000'
EARLIEST_DEPARTURE_TIME = '07:20:00'

In [ ]:
ARRIVAL_TIME = dt.datetime.strptime(ARRIVAL_TIME, '%H:%M:%S')
EARLIEST_DEPARTURE_TIME = dt.datetime.strptime(EARLIEST_DEPARTURE_TIME, '%H:%M:%S')
table_graph = table_graph.loc[(table_graph[TRAVEL_DAY]==1) & (table_graph['arrival_time']<=ARRIVAL_TIME)  & (table_graph['departure_time']>=EARLIEST_DEPARTURE_TIME)].copy() 
table_graph.drop(columns=['monday', 'tuesday', 'wednesday', 'thursday', 'friday'], inplace=True)

In [ ]:
#routing algorithm
def routing(df_graph, arrival_stop_id, arrival_time, proba_threshold):
    
    df_graph.sort_values(by=['src_departure_time'], ascending=False, inplace=True) # ordering the edges in descending order according to their starting time
    
    transfer_time = dt.timedelta(minutes=2) # minimal transfer time between two different transports at the same location
    distinct_stops = pd.concat([df_graph['src_stop_id'],df_graph['dst_stop_id']]).unique() # retrieving the nodes of the graph from the edges
    path = dict()  # dict that takes as key a departure stop_id and as value a list [departure_time, arrival_time, arrival stop_id, transport_ID, proba_sucess, transport mode]
    
    ### 1. Put default list for each stop ###
    
    for stop_id in distinct_stops:
        
        if (stop_id == arrival_stop_id):
            path[stop_id] = [arrival_time, arrival_time, arrival_stop_id, None, 1, None]
       
        else:
            path[stop_id] = [dt.datetime.strptime('00:00:01', '%H:%M:%S'), dt.datetime.strptime('00:00:01', '%H:%M:%S'), None, None, None, None]   
            
    ### 2. Scans each edge to update the list of each stop ###
    
    progress_bar = tqdm(total=len(df_graph), desc='Processing', unit='iteration')
    for i, edge in df_graph.iterrows():
        
        u = edge['src_stop_id']  # stop_id of departure
        v = edge['dst_stop_id']  # stop_id of arrival
        t = edge['src_departure_time']  # instant of departure
        delta = dt.timedelta(minutes=edge['duration'])  # duration of the journey
        trip_id = edge['trip_id']  # trip_id
        vehicule = edge['vehicule']  # vehicule
        transfer_time = dt.timedelta(minutes=2) #redefined here since it might become 0 if we walk or we do not change transport type cf. what follows
        
        if ((trip_id == path[v][3]) or (vehicule=='foot')):  # checking if the transport ID of the edge (connection) is the same as the one we are taking afterwards (from v) 
            transfer_time = dt.timedelta(minutes=0)  # no transfer time between two same transport id
            if ((t + delta) <= path[v][0]):  # checking if the connection makes us arrive at stop v before the next transport departs from stop v
                if (t > path[u][0]):  # updating the path only if the connection makes us depart from u later (we want the latest departure time)
                    path[u][0] = t 
                    path[u][1] = t + delta
                    path[u][2] = v
                    path[u][3] = trip_id
                    path[u][4] = path[v][4]
                    path[u][5] = vehicule

        else :  # changing transport ID
            if v == arrival_stop_id: # no need to add transfer time if we arrive at the final destination
                transfer_time = dt.timedelta(minutes=0)
     
            if ((t + delta + transfer_time) <= path[v][0]): # same logic as before but making sure that we keep some time (transfer_time) between the two connections
                if (t > path[u][0]):
                    delay_max = path[v][0] - (t + delta + transfer_time)  # maximum delay time
                    proba_delay = delay_proba(delay_max.seconds // 60 % 60, t + delta, vehicule, v) # computing the probability to make the connection based on this maximum delay time
                    proba_success = 1 - proba_delay
                    if (proba_success*path[v][4] > proba_threshold/100): #updating the path if the probability is sufficiently high
                        path[u][0] = t
                        path[u][1] = t + delta
                        path[u][2] = v
                        path[u][3] = trip_id
                        path[u][4] = proba_success*path[v][4]
                        path[u][5] = vehicule
        
        progress_bar.update(1)
        
    progress_bar.close()
    return path

In [ ]:
# Probabilities

from scipy.sparse import hstack
import pickle

# loading models
with open(f'../models/transport_type_ohe.pkl','rb') as f:
    transport_type_ohe = pickle.load(f)

with open(f'../models/stop_id_ohe.pkl','rb') as f:
    stop_id_ohe = pickle.load(f)

with open(f'../models/scaler.pkl','rb') as f:
    scaler = pickle.load(f)

with open(f'../models/log.pkl','rb') as f:
    log = pickle.load(f)

# Give the bin for the delay
def label_bin(x):
    bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30]
    bins_id = range(len(bins) + 1)
    for i, b in enumerate(bins):
        if x <= b:
            return bins_id[i]
    return bins_id[-1]

def delay_proba(delay_max, arrival_time_shedule, transport_type, stop_id):
        
    try:
        df = pd.DataFrame(columns=['arrival_time_shedule', 'transport_type', 'stop_id'])
        df.loc[0] = [arrival_time_shedule, str(transport_type), str(stop_id)]

        df.arrival_time_shedule = pd.to_datetime(df.arrival_time_shedule, format='%d.%m.%Y %H:%M')

        # Compute the predictors
        df["minute"] = df.arrival_time_shedule.dt.minute
        df["hour"] = df.arrival_time_shedule.dt.hour
        df["month"] = df.arrival_time_shedule.dt.month
        df["year"] = df.arrival_time_shedule.dt.year
        df["week_of_year"] = df.arrival_time_shedule.dt.isocalendar().week
        df["day_of_year"] = df.arrival_time_shedule.dt.day_of_year
        df["day_of_week"] = df.arrival_time_shedule.dt.day_of_week

        inputCols = ["minute", "hour", "month", "year", "week_of_year", "day_of_year", "day_of_week"]
        ohe_cols = ["transport_type", "stop_id"]
        X = df[inputCols].to_numpy(dtype=int)
        
        # Scale data
        X = scaler.transform(X)
        
        # One hot encode transport type
        transformed = transport_type_ohe.transform(df[[ohe_cols[0]]])
        X = hstack((X, transformed), format='csr')

        # One hot encode stop id
        transformed = stop_id_ohe.transform(df[[ohe_cols[1]]])
        X = hstack((X, transformed), format='csr')

        # Prediect the probability distribution
        prob = log.predict_proba(X)[0]

        # Get the delay bin
        delay_bin = label_bin(delay_max)

        # The probability of arrival time below delay_max is the sum of the prediected probability for delay <= delay_max
        total_proba = sum(prob[:delay_bin + 1])

        return total_proba
    
    # If the model fail for some raison
    except:
        return min(0.99 ** (10 - delay_max), 1)

In [ ]:
# Exemple
delay_proba(5, '02.01.2022 15:30', 'Zug', '8503006')

In [ ]:
path = routing(df_graph, DESTINATION, ARRIVAL_TIME, PROBA_THRESHOLD)

In [ ]:
path_good = pd.DataFrame.from_dict(path, orient='index', dtype=None, columns=['departure_time','arrival_time','destination','trip_id','proba', 'vehicule'])
path_good

In [ ]:
path_good.dropna(subset = ['destination', 'trip_id'], inplace=True)  # => PAS NORMAL D AVOIR DES TRIP_ID NONE => à checker, commande en attendant
path_good['departure_time'] = path_good['departure_time'].apply(lambda x: x.time())
path_good['arrival_time'] = path_good['arrival_time'].apply(lambda x: x.time())
#path_good = path_good[path_good['destination'] is not None]
path_good.sort_values(by = ['departure_time'])

In [ ]:
# Get the final path to follow

final_path = [[DEPARTURE, path_good.loc[DEPARTURE].departure_time, path_good.loc[DEPARTURE].arrival_time, path_good.loc[DEPARTURE].destination, path_good.loc[DEPARTURE].trip_id, path_good.loc[DEPARTURE].vehicule]]
destination = path_good.loc[DEPARTURE].destination

for i in range(len(path_good)):
    try:
        final_path.append([destination, path_good.loc[destination].departure_time, path_good.loc[destination].arrival_time, path_good.loc[destination].destination, path_good.loc[destination].trip_id, path_good.loc[DEPARTURE].vehicule])
        destination = path_good.loc[destination].destination
    except:
        print("There is no path available to go to the final destination. This is the beginning of the path:")
        break

df_final_path = pd.DataFrame(final_path, columns=['departure', 'departure_time', 'arrival_time', 'destination', 'trip_id', 'vehicule'])
df_final_path.groupby(['trip_id', 'vehicule']).agg(list)